<a href="https://colab.research.google.com/github/kartikrupal/Hate-Speech/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Necessary Libraries

In [43]:
import pandas as pd
import numpy as np
import spacy
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE

Loading Dataset

In [44]:
# Install this in your environment if not already done
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

# File inside the dataset
file_path = "labeled_data.csv"

# Load the dataset
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mrmorj/hate-speech-and-offensive-language-dataset",
    file_path
)

# Display the first few rows
print(df.head())


/tmp/ipython-input-44-79205389.py:11: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [45]:
df.head(15)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [46]:
df.columns

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')

Dataset Cleaning

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [48]:
df.isnull().sum()

,0
Unnamed: 0,0
count,0
hate_speech,0
offensive_language,0
neither,0
class,0
tweet,0


In [49]:
df.drop(columns=['count','hate_speech','offensive_language','neither','Unnamed: 0'],inplace=True)

In [50]:
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


removing all the special characters

In [51]:
df['New_tweet']=df['tweet'].str.replace(r'[^a-zA-Z]',' ',regex=True)

In [52]:
df

,class,tweet,New_tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldn t...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats cold tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C G Anderson viva based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you...
...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,you s a muthaf in lie LifeAsKing ...
24779,2,"you've gone and broke the wrong heart baby, an...",you ve gone and broke the wrong heart baby an...
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [53]:
df['New_tweet'].iloc[5]

'                    T Madison x  The shit just blows me  claim you so faithful and down for somebody but still fucking with hoes                              '

In [54]:
df['New_tweet']=df['New_tweet'].str.replace(r'[\s]+',' ',regex=True)

In [55]:
df

,class,tweet,New_tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldn t comp...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats cold tyga dwn bad for cuffi...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbaby life You ever ...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C G Anderson viva based she look like a tr...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you hear about me ...
...,...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,you s a muthaf in lie LifeAsKing Pearls corey ...
24779,2,"you've gone and broke the wrong heart baby, an...",you ve gone and broke the wrong heart baby and...
24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like I aint fu...
24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [56]:
df['New_tweet'].iloc[5]

' T Madison x The shit just blows me claim you so faithful and down for somebody but still fucking with hoes '

In [57]:
df.drop(columns=['tweet'],inplace=True)

In [58]:
df

,class,New_tweet
0,2,RT mayasolovely As a woman you shouldn t comp...
1,1,RT mleew boy dats cold tyga dwn bad for cuffi...
2,1,RT UrKindOfBrand Dawg RT sbaby life You ever ...
3,1,RT C G Anderson viva based she look like a tr...
4,1,RT ShenikaRoberts The shit you hear about me ...
...,...,...
24778,1,you s a muthaf in lie LifeAsKing Pearls corey ...
24779,2,you ve gone and broke the wrong heart baby and...
24780,1,young buck wanna eat dat nigguh like I aint fu...
24781,1,youu got wild bitches tellin you lies


***NLP PROCESSING***

In [59]:
import spacy

In [60]:
nlp=spacy.load('en_core_web_sm')


In [61]:
def lemmatization(text):
  doc=nlp(text)
  lemmatized = [word.lemma_ for word in doc]
  return " ".join(lemmatized)


In [62]:
text="kartik is in 3rd year"
lemmatization(text)

'kartik be in 3rd year'

In [63]:
df['processed_text']=df['New_tweet'].apply(lemmatization)

In [64]:
df.head(10)

,class,New_tweet,processed_text
0,2,RT mayasolovely As a woman you shouldn t comp...,RT mayasolovely as a woman you shouldn t com...
1,1,RT mleew boy dats cold tyga dwn bad for cuffi...,RT mleew boy dat cold tyga dwn bad for cuffi...
2,1,RT UrKindOfBrand Dawg RT sbaby life You ever ...,RT UrKindOfBrand Dawg RT sbaby life you ever...
3,1,RT C G Anderson viva based she look like a tr...,RT C G Anderson viva base she look like a tr...
4,1,RT ShenikaRoberts The shit you hear about me ...,RT ShenikaRoberts the shit you hear about I ...
5,1,T Madison x The shit just blows me claim you ...,T Madison x the shit just blow I claim you s...
6,1,BrighterDays I can not just sit up and HATE o...,BrighterDays I can not just sit up and HATE ...
7,1,selfiequeenbri cause I m tired of you big bit...,selfiequeenbri cause I m tired of you big bi...
8,1,amp you might not get ya bitch back amp thats...,amp you might not get ya bitch back amp that...
9,1,rhythmixx hobbies include fighting Mariam bitch,rhythmixx hobby include fight Mariam bitch


In [65]:
def stop_removal(text):
  doc=nlp(text)
  filtered_tokens = [token.text for token in doc if not token.is_stop]
  return " ".join(filtered_tokens)

In [ ]:
df['processed_text_2']=df['processed_text'].apply(stop_removal)

In [ ]:
df

In [ ]:
df.drop(columns=['New_tweet','processed_text'],inplace=True)

In [ ]:
df

In [ ]:
df = df.rename(columns={'processed_text_2': 'content'})
df

In [ ]:
df['content'].iloc[67]

**ONE HOT ENCODING**

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size=10000
one_hot_representation=[ one_hot(words,vocab_size) for words in df['content']]

In [ ]:
for i in range(6):
  print(df['content'].iloc[i])

In [ ]:
for i in range(6):
  print(one_hot_representation[i])


In [ ]:
sentence_length= 20
embedded_tweet=pad_sequences(one_hot_representation,padding='pre',maxlen=sentence_length)

In [ ]:
for i in range(6):
  print(embedded_tweet[i])


In [ ]:
X=np.array(embedded_tweet)
y=np.array(df['class'])
df['class'].value_counts()

Fixing the inconsistent data oversampling

In [ ]:
smote=SMOTE(sampling_strategy='minority')
X,y=smote.fit_resample(X,y)


***Model Building***

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X.shape,X_train.shape,X_test.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
dimension =50
model=keras.Sequential([
    layers.Embedding(vocab_size,dimension,input_length=sentence_length),
    #lstm
    layers.LSTM(128,return_sequences=True),
    layers.LSTM(64,return_sequences=True),
    layers.LSTM(64),

    layers.Dense(3,activation="softmax")
    ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=20,batch_size=32,validation_data=(X_test,y_test))

In [ ]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Accuracy Of Model:{accuracy*100}")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create and configure the tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")

# Fit the tokenizer on training data (this is just a placeholder; use your actual data)
tokenizer.fit_on_texts([
    "This is an example sentence.",
    "Another sentence to train the tokenizer.",
    "Add more training data based on your dataset."
])

# Input text for prediction
text = ["Go back to where you came from. You don't belong here."]

# Convert input to sequences
seq = tokenizer.texts_to_sequences(text)

# Pad the sequence
padded = pad_sequences(seq, maxlen=20)

# Predict using your model
prediction = model.predict(padded)
pred=np.argmax((prediction),axis=-1)
if(pred==0):
 print("Hate Speech")
elif(pred==1):
  print("Offensive")
else:
  print("Neither")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
prediction=model.predict(X_test)
# y_test: true labels
# y_pred_prob: model's predicted probabilities (from model.predict)
# Get class predictions (e.g., for softmax output)
y_pred = np.argmax(prediction, axis=1)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
labels = ['Hate Speech', 'Offensive', 'Neither']

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=labels))


In [ ]:
# --- After Data Cleaning and Before One-Hot Encoding ---

from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

# Define constants
VOCAB_SIZE = 10000
MAX_LEN = 20

# 1. Create and fit the tokenizer on your processed text
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

# Save the tokenizer for later use in the web app
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 2. Convert text to sequences and pad them
sequences = tokenizer.texts_to_sequences(df['content'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_LEN, padding='pre')

# --- Then, continue with SMOTE and model training as you did ---
# X = np.array(padded_sequences)
# y = np.array(df['class'])
# ... smote, train_test_split, etc. ...

# 3. After training is complete, save the final model
model.save('hate_speech_model.h5')

print("✅ Model and Tokenizer have been saved successfully!")